In [58]:
import requests
import xmltodict
import urllib, urllib.request
import feedparser
from datetime import date
from xml.etree import ElementTree
import time
import arxiv
import pandas as pd

BASE = 'http://export.arxiv.org/api'

In [ ]:
import arxiv

search = arxiv.Search(
    query='a',
    max_results = 300000,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

# for result in search.results():
#     print(result.title, result.published)

In [ ]:
results = []
for result in search.results():
    results.append(result)
    if len(results) == 100:
        break

In [ ]:
list(search.results())

In [ ]:
url = 'http://export.arxiv.org/api/query?search_query=all:a+OR+all:the'
url = 'http://export.arxiv.org/oai2?verb=ListRecords&set=math&from=2015-01-01&until=2015-01-31&metadataPrefix=arXiv'
data = urllib.request.urlopen(url)
#print(data.read().decode('utf-8'))
parsed = feedparser.parse(data)
parsed

In [ ]:
# data_test = urllib.request.urlopen('http://export.arxiv.org/oai2?verb=ListRecords&set=math&from=2015-01-01&until=2015-01-31&metadataPrefix=arXiv')

In [ ]:
# feedparser.parse(data_test)

In [145]:
# get math stuff from 2015
# response = requests.get('http://export.arxiv.org/oai2?verb=ListRecords&set=math&from=2015-01-01&until=2015-01-31&metadataPrefix=arXiv')

# get EVERYTHING
# response = requests.get('http://export.arxiv.org/oai2?verb=ListRecords&set=math&metadataPrefix=arXiv&until=2007-05-25')
# print(response.text)
# response2 = requests.get('http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=5807382|38001')
# print(response2.text)
dict = xmltodict.parse(response2.text)
if dict['OAI-PMH']['ListRecords']['resumptionToken'].get('#text'):
    print('Still more')
else:
    print('Done!')

dict1 = xmltodict.parse(response.text)
dict1.update(dict)
# dict = json.loads(json.dumps(dict))
type(dict['OAI-PMH']['ListRecords']['record'])


Done!


list

In [106]:
# print(response.text)
# response2 = requests.get('http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=5807359|1001')
# print(response2.text)
dict = xmltodict.parse(response.text)
dict['OAI-PMH']['ListRecords']['resumptionToken']['#text']

'5807359|1001'

In [104]:
# for processing giant dict at very end

import json
dict = json.loads(json.dumps(dict))
dict['OAI-PMH']['ListRecords']['resumptionToken']['#text']
# dict['OAI-PMH']['resumptionToken']['#text']
records = dict['OAI-PMH']['ListRecords']['record']
df1 = pd.DataFrame(records)
df2 = pd.DataFrame(list(df1['metadata']))
#records
df2
records
df1['metadata'][0]['arXiv']


{'@xmlns': 'http://arxiv.org/OAI/arXiv/',
 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@xsi:schemaLocation': 'http://arxiv.org/OAI/arXiv/ http://arxiv.org/OAI/arXiv.xsd',
 'id': '0704.0001',
 'created': '2007-04-02',
 'updated': '2007-07-24',
 'authors': {'author': [{'keyname': 'Balázs', 'forenames': 'C.'},
   {'keyname': 'Berger', 'forenames': 'E. L.'},
   {'keyname': 'Nadolsky', 'forenames': 'P. M.'},
   {'keyname': 'Yuan', 'forenames': 'C. -P.'}]},
 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
 'categories': 'hep-ph',
 'comments': '37 pages, 15 figures; published version',
 'report-no': 'ANL-HEP-PR-07-12',
 'journal-ref': 'Phys.Rev.D76:013009,2007',
 'doi': '10.1103/PhysRevD.76.013009',
 'abstract': 'A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark

In [ ]:
date = '2022-03-03'
start = 0
max = 100
papers = []
math_query = """
cat:math.ac+OR+cat:math.ag+OR+cat:math.ap+OR+cat:math.at+OR+cat:math.ca+OR+cat:math.co+OR+
cat:math.ct+OR+cat:math.cv+OR+cat:math.dg+OR+cat:math.ds+OR+cat:math.fa+OR+cat:math.gm+OR+
cat:math.gn+OR+cat:math.gr+OR+cat:math.gt+OR+cat:math.ho+OR+cat:math.it+OR+cat:math.kt+OR+
cat:math.lo+OR+cat:math.mg+OR+cat:math.mp+OR+cat:math.na+OR+cat:math.nt+OR+cat:math.oa+OR+
cat:math.oc+OR+cat:math.pr+OR+cat:math.qa+OR+cat:math.ra+OR+cat:math.rt+OR+cat:math.sg+OR+
cat:math.sp+OR+cat:math.st
""".replace('\n', '')

while (date == '2022-03-03'):
    print(f'Results {start} - ' + str(start + max))
    query = f'/query?search_query={math_query}&start={start}&max_results={max}&sortBy=submittedDate'

    res = data = urllib.request.urlopen(BASE + query)
    parsed = feedparser.parse(res)

    for entry in parsed.entries:
        paper = {}
        paper['date'] = entry.published
        paper['title'] = entry.title
        paper['first_author'] = entry.author
        paper['summary'] = entry.summary
        paper['primary_category'] = entry.arxiv_primary_category['term']
        papers.append(paper)
        date = paper['date'][0:10]

    start += max


    if not papers:
        break

    time.sleep(3)

In [ ]:
print(len(papers))
papers

In [ ]:
# query = f'/query?search_query={fuck_me}&start=0&max_results=10&sortBy=submittedDate'
# res = data = urllib.request.urlopen(BASE + query)
# parsed = feedparser.parse(res)
parsed.entries[0].arxiv_primary_category['term']

In [ ]:
"""
This sample script illustrates a basic arXiv api call
followed by parsing of the results using the 
feedparser python module.

Please see the documentation at 
http://export.arxiv.org/api_help/docs/user-manual.html
for more information, or email the arXiv api 
mailing list at arxiv-api@googlegroups.com.

urllib is included in the standard python library.
feedparser can be downloaded from http://feedparser.org/ .

Author: Julius B. Lucks

This is free software.  Feel free to do what you want
with it, but please play nice with the arXiv API!
"""

import urllib
import feedparser

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'all:electron' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 5

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

# Opensearch metadata such as totalResults, startIndex, 
# and itemsPerPage live in the opensearch namespase.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

# perform a GET request using the base_url and query
response = urllib.urlopen(base_url+query).read()

# parse the response using feedparser
feed = feedparser.parse(response)

# print out feed information
print ('Feed title: %s' % feed.feed.title)
print 'Feed last updated: %s' % feed.feed.updated

# print opensearch metadata
print 'totalResults for this query: %s' % feed.feed.opensearch_totalresults
print 'itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage
print 'startIndex for this query: %s'   % feed.feed.opensearch_startindex

# Run through each entry, and print out information
for entry in feed.entries:
    print 'e-print metadata'
    print 'arxiv-id: %s' % entry.id.split('/abs/')[-1]
    print 'Published: %s' % entry.published
    print 'Title:  %s' % entry.title
    
    # feedparser v4.1 only grabs the first author
    author_string = entry.author
    
    # grab the affiliation in <arxiv:affiliation> if present
    # - this will only grab the first affiliation encountered
    #   (the first affiliation for the first author)
    # Please email the list with a way to get all of this information!
    try:
        author_string += ' (%s)' % entry.arxiv_affiliation
    except AttributeError:
        pass
    
    print 'Last Author:  %s' % author_string
    
    # feedparser v5.0.1 correctly handles multiple authors, print them all
    try:
        print 'Authors:  %s' % ', '.join(author.name for author in entry.authors)
    except AttributeError:
        pass

    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == 'alternate':
            print 'abs page link: %s' % link.href
        elif link.title == 'pdf':
            print 'pdf link: %s' % link.href
    
    # The journal reference, comments and primary_category sections live under 
    # the arxiv namespace
    try:
        journal_ref = entry.arxiv_journal_ref
    except AttributeError:
        journal_ref = 'No journal ref found'
    print 'Journal reference: %s' % journal_ref
    
    try:
        comment = entry.arxiv_comment
    except AttributeError:
        comment = 'No comment found'
    print 'Comments: %s' % comment
    
    # Since the <arxiv:primary_category> element has no data, only
    # attributes, feedparser does not store anything inside
    # entry.arxiv_primary_category
    # This is a dirty hack to get the primary_category, just take the
    # first element in entry.tags.  If anyone knows a better way to do
    # this, please email the list!
    print 'Primary Category: %s' % entry.tags[0]['term']
    
    # Lets get all the categories
    all_categories = [t['term'] for t in entry.tags]
    print 'All Categories: %s' % (', ').join(all_categories)
    
    # The abstract is in the <summary> element
    print 'Abstract: %s' %  entry.summary

In [4]:
import os.path
BASE_DIR = os.path.dirname(os.path.abspath('play.ipynb'))
db_path = os.path.join(BASE_DIR, 'arxiv.db')

In [5]:
os.getcwd()
BASE_DIR = os.path.dirname(os.path.abspath('play.ipynb'))

'c:\\Users\\flame\\Documents\\Projects\\arxiv-oai-scraper\\src\\arxiv.db'

In [7]:
import sqlite3
con = sqlite3.connect(db_path)
cursor = con.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('papers',)]
